In [94]:
import sympy

def get_M(a, b, b0):
    X = a-b
    X = X / sympy.sqrt( X.dot(X) )
    Z = X.cross(b0-b)
    Z = Z / sympy.sqrt( Z.dot(Z) )
    Y = Z.cross(X)
    M=sympy.Matrix([[X[0],Y[0],Z[0]],[X[1],Y[1],Z[1]],[X[2],Y[2],Z[2]]])
    return sympy.simplify(M)

def get_W(dist, ca,ct,sa,st):
    W = sympy.Matrix([
         dist * ca,
         dist * sa * ct,
         dist * sa * st
    ])
    return W

def build_acc_water( a, b, b0, dist, ca,ct,sa,st ):
    M=get_M(a, b, b0)
    wat_loc = get_W(dist, ca,ct,sa,st)
    water = M*wat_loc + a
    return water

ax,ay,az = sympy.symbols('A[0],A[1],A[2]')
A = sympy.Matrix([ax,ay,az])
bx,by,bz = sympy.symbols('B[0],B[1],B[2]')
B = sympy.Matrix([bx,by,bz])
b0x,b0y,b0z = sympy.symbols('B0[0],B0[1],B0[2]')
B0 = sympy.Matrix([b0x,b0y,b0z])
d,ca,ct,sa,st = sympy.symbols('dist,cos_a,cos_t,sin_a,sin_t') # cos_a=cos(pi-a)
sympy.init_printing(False)

acc_wat = build_acc_water(A,B,B0,d,ca,ct,sa,st)

In [102]:
a_m_b,a_m_b2,b0_m_b = sympy.symbols('a_m_b,a_m_b2,b0_m_b')
ab_x,ab_y,ab_z = sympy.symbols('AB[0],AB[1],AB[2]')
bb0_x,bb0_y,bb0_z = sympy.symbols('B0B[0],B0B[1],B0B[2]')
abxb0b_x,abxb0b_y,abxb0b_z = sympy.symbols('ABXB0B[0],ABXB0B[1],ABXB0B[2]')

Derivs={}
for s in (ax,ay,az,bx,by,bz,b0x,b0y,b0z):
    Derivs[s] = sympy.diff(acc_wat,s)
    Derivs[s] = Derivs[s].subs(sympy.sqrt((ax-bx)**2+(ay-by)**2+(az-bz)**2), a_m_b)
    Derivs[s] = Derivs[s].subs(a_m_b**2, a_m_b2)
    Derivs[s] = Derivs[s].subs(sympy.sqrt((b0x-bx)**2+(b0y-by)**2+(b0z-bz)**2), b0_m_b)
    Derivs[s] = Derivs[s].subs((ax - bx), ab_x)
    Derivs[s] = Derivs[s].subs((ay - by), ab_y)
    Derivs[s] = Derivs[s].subs((az - bz), ab_z)
    Derivs[s] = Derivs[s].subs((b0x - bx), b0b_x)
    Derivs[s] = Derivs[s].subs((b0y - by), b0b_y)
    Derivs[s] = Derivs[s].subs((b0z - bz), b0b_z)
    Derivs[s] = Derivs[s].subs(ab_y*bb0_z - ab_z*bb0_y, abxb0b_x)
    Derivs[s] = Derivs[s].subs(ab_z*bb0_x - ab_x*bb0_z, abxb0b_y)
    Derivs[s] = Derivs[s].subs(ab_x*bb0_y - ab_y*bb0_x, abxb0b_z)
    Derivs[s] = Derivs[s].subs(abxb0b_x**2 +abxb0b_y**2+abxb0b_z**2, 'abxb0b2')

In [112]:
Derivs[b0z]

Matrix([
[-ABXB0B[0]*dist*sin_a*sin_t*(ABXB0B[0]*(2*A[1] - 2*B[1]) - ABXB0B[1]*(2*A[0] - 2*B[0]))/(2*a_m_b*abxb0b2*sqrt(abxb0b2/a_m_b2)) - AB[0]*AB[2]*cos_t*dist*sin_a/(a_m_b2*sqrt(abxb0b2/a_m_b2)) + AB[1]*dist*sin_a*sin_t/(a_m_b*sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*(ABXB0B[0]*(2*A[1] - 2*B[1]) - ABXB0B[1]*(2*A[0] - 2*B[0]))*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])/(2*a_m_b2*abxb0b2*sqrt(abxb0b2/a_m_b2))],
[-ABXB0B[1]*dist*sin_a*sin_t*(ABXB0B[0]*(2*A[1] - 2*B[1]) - ABXB0B[1]*(2*A[0] - 2*B[0]))/(2*a_m_b*abxb0b2*sqrt(abxb0b2/a_m_b2)) - AB[0]*dist*sin_a*sin_t/(a_m_b*sqrt(abxb0b2/a_m_b2)) - AB[1]*AB[2]*cos_t*dist*sin_a/(a_m_b2*sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])*(ABXB0B[0]*(2*A[1] - 2*B[1]) - ABXB0B[1]*(2*A[0] - 2*B[0]))/(2*a_m_b2*abxb0b2*sqrt(abxb0b2/a_m_b2))],
[                                             -ABXB0B[2]*dist*sin_a*sin_t*(ABXB0B[0]*(2*A[1] - 2*B[1]) - ABXB0B[1]*(2*A[0] - 2*B[0]))/(2*a_m_b*abxb0b2*sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin

In [137]:
ds = [Derivs[s] for s in (ax,ay,az,bx,by,bz,b0x,b0y,b0z)]
repl,red = sympy.cse(ds, symbols=sympy.utilities.iterables.numbered_symbols(start=101), 
                     ignore=(d,ca,ct,sa,st, a_m_b,a_m_b2,b0_m_b,ab_x,ab_y,ab_z,bb0_x,bb0_y,bb0_z,
                             axb_x,axb_y,axb_z))
keysS = []
valsS = []
for key, val in repl:
    keysS.append(str(key))
    valsS.append(str(val))

lhsS = []
rhsS = []
for i in range(3):
    for j in range(3):
        for k in range(3):
            if (red[3*i+j][k]!=0):
                lhsS.append('dwda['+str(i)+','+str(j)+','+str(k)+']')
                rhsS.append(str(red[3*i+j][k]))

for i in range(len(keysS)):
    print (keysS[i],'=',valsS[i])
for i in range(len(lhsS)):
    print (lhsS[i],'=',rhsS[i])

x101 = 2*A[0]
x102 = -x101
x103 = 2*B[0]
x104 = x102 + x103
x105 = 1/abxb0b2
x106 = 2*B0[1]
x107 = 2*B[1]
x108 = -x107
x109 = 2*B0[2]
x110 = 2*B[2]
x111 = -x110
x112 = x109 + x111
x113 = -ABXB0B[1]*x112 + ABXB0B[2]*(x106 + x108)
x114 = 2*A[1]
x115 = -x114
x116 = x107 + x115
x117 = 2*B0[0]
x118 = x103 - x117
x119 = ABXB0B[0]*x112 + ABXB0B[2]*x118
x120 = 2*A[2]
x121 = x110 - x120
x122 = -x106
x123 = ABXB0B[0]*(x107 + x122) - ABXB0B[1]*x118
x124 = A[1] - B0[1]
x125 = A[2] - B0[2]
x126 = x101 - x103
x127 = -x109 + x120
x128 = -ABXB0B[1]*x127 + ABXB0B[2]*(x114 + x122)
x129 = -ABXB0B[2]
x130 = -A[0] + B0[0]
x131 = x108 + x114
x132 = x102 + x117
x133 = ABXB0B[0]*x127 + ABXB0B[2]*x132
x134 = -A[1] + B0[1]
x135 = x111 + x120
x136 = ABXB0B[0]*(x106 + x115) - ABXB0B[1]*x132
x137 = -ABXB0B[1]*x121 + ABXB0B[2]*x116
x138 = ABXB0B[0]*x121 + ABXB0B[2]*x126
x139 = ABXB0B[0]*x131 - ABXB0B[1]*x126
dwda[0,0,0] = -ABXB0B[0]*AB[0]*dist*sin_a*sin_t/(a_m_b**3*sqrt(abxb0b2/a_m_b2)) + ABXB0B[0]*a_m_b*dist*sin_a

In [148]:
import numpy
def dWref(A, B, B0, dist, angle, torsion):
    dWdA = numpy.zeros((3, 3, 3))
    x101 = -B[2]
    x102 = A[2] + x101
    x103 = -B[0]
    x104 = B0[0] + x103
    x105 = A[0] + x103
    x106 = x105 * x105
    x107 = -B[1]
    x108 = A[1] + x107
    x109 = x108 * x108
    x110 = x102 * x102
    x111 = x106 + x109 + x110
    x112 = 1 / numpy.sqrt(x111)
    x113 = x104 * x112
    x114 = B0[2] + x101
    x115 = x112 * x114
    x116 = x102 * x113 - x105 * x115
    x117 = B0[1] + x107
    x118 = x112 * x117
    x119 = x105 * x118 - x108 * x113
    x120 = -x102 * x118 + x108 * x115
    x121 = x116 * x116 + x119 * x119 + x120 * x120
    x122 = 1 / numpy.sqrt(x121)
    x123 = x111 ** (-1.5)
    x124 = x123 * (-A[0] + B[0])
    x125 = x122 * x124
    x126 = x102 * x125
    x127 = x108 * x125
    x128 = x108 * x112
    x129 = x105 * x124
    x130 = x117 * x129
    x131 = x104 * x124
    x132 = x108 * x131
    x133 = x122 * (x118 + x130 - x132)
    x134 = -x115
    x135 = x102 * x131
    x136 = x114 * x129
    x137 = x134 + x135 - x136
    x138 = x102 * x112
    x139 = x122 * x138
    x140 = x116 * x138
    x141 = x121 ** (-1.5)
    x142 = x108 * x114
    x143 = x124 * x142
    x144 = x102 * x117
    x145 = x124 * x144
    x146 = x120 / 2
    x147 = 2 * x118
    x148 = x119 / 2
    x149 = 2 * x115
    x150 = -x149
    x151 = x116 / 2
    x152 = x141 * (
        -x146 * (2 * x143 - 2 * x145)
        - x148 * (2 * x130 - 2 * x132 + x147)
        - x151 * (2 * x135 - 2 * x136 + x150)
    )
    x153 = x119 * x128
    x154 = torsion
    x155 = numpy.pi - angle
    x156 = dist * numpy.sin(x155)
    x157 = x156 * numpy.cos(x154)
    x158 = dist * numpy.cos(x155)
    x159 = x143 - x145
    x160 = x156 * numpy.sin(x154)
    x161 = x122 * x160
    x162 = x152 * x160
    x163 = x112 * x158
    x164 = x163 + 1
    x165 = x124 * x158
    x166 = x112 * x122
    x167 = x119 * x166
    x168 = x122 * x129
    x169 = x105 * x112
    x170 = x119 * x169
    x171 = x120 * x138
    x172 = x116 * x166
    x173 = -x172
    x174 = x122 * x128
    x175 = x122 * x169
    x176 = x120 * x128
    x177 = x116 * x169
    x178 = x123 * (-A[1] + B[1])
    x179 = x105 * x178
    x180 = x142 * x178
    x181 = x144 * x178
    x182 = x115 + x180 - x181
    x183 = x114 * x179
    x184 = x104 * x178
    x185 = x102 * x184
    x186 = 2 * x113
    x187 = -x186
    x188 = x117 * x179
    x189 = x108 * x184
    x190 = x141 * (
        -x146 * (x149 + 2 * x180 - 2 * x181)
        - x148 * (x187 + 2 * x188 - 2 * x189)
        - x151 * (-2 * x183 + 2 * x185)
    )
    x191 = x160 * x190
    x192 = -x167
    x193 = x122 * x178
    x194 = x102 * x193
    x195 = x108 * x193
    x196 = -x183 + x185
    x197 = -x113
    x198 = x188 - x189 + x197
    x199 = x122 * x179
    x200 = x158 * x178
    x201 = x120 * x166
    x202 = x123 * (-A[2] + B[2])
    x203 = x105 * x202
    x204 = -x118
    x205 = x142 * x202
    x206 = x144 * x202
    x207 = x204 + x205 - x206
    x208 = x117 * x203
    x209 = x104 * x202
    x210 = x108 * x209
    x211 = x114 * x203
    x212 = x102 * x209
    x213 = -x147
    x214 = x141 * (
        -x146 * (2 * x205 - 2 * x206 + x213)
        - x148 * (2 * x208 - 2 * x210)
        - x151 * (x186 - 2 * x211 + 2 * x212)
    )
    x215 = x160 * x214
    x216 = x122 * x202
    x217 = x102 * x216
    x218 = x108 * x216
    x219 = x208 - x210
    x220 = x113 - x211 + x212
    x221 = x158 * x202
    x222 = -x201
    x223 = x122 * x203
    x224 = -x163
    x225 = x106 * x123
    x226 = x105 * x123
    x227 = x142 * x226
    x228 = x144 * x226
    x229 = x227 - x228
    x230 = 2 * x227
    x231 = 2 * x228
    x232 = 2 * x128
    x233 = x117 * x225
    x234 = x104 * x226
    x235 = x108 * x234
    x236 = 2 * x138
    x237 = x114 * x225
    x238 = x102 * x234
    x239 = x141 * (
        -x146 * (x230 - x231)
        - x148 * (x213 + x232 + 2 * x233 - 2 * x235)
        - x151 * (x149 - x236 - 2 * x237 + 2 * x238)
    )
    x240 = x160 * x239
    x241 = x102 * x226
    x242 = x122 * x241
    x243 = x116 * x242
    x244 = x108 * x226
    x245 = x122 * x244
    x246 = x119 * x245
    x247 = x115 - x138 - x237 + x238
    x248 = x128 + x204 + x233 - x235
    x249 = x158 * x244
    x250 = x122 * x225
    x251 = x158 * x241
    x252 = x109 * x123
    x253 = x114 * x252
    x254 = x108 * x123
    x255 = x144 * x254
    x256 = x134 + x138 + x253 - x255
    x257 = x102 * x254
    x258 = x104 * x257
    x259 = 2 * x258
    x260 = 2 * x169
    x261 = x104 * x252
    x262 = x117 * x244
    x263 = x141 * (
        -x146 * (x150 + x236 + 2 * x253 - 2 * x255)
        - x148 * (x186 - x260 - 2 * x261 + 2 * x262)
        - x151 * (-x230 + x259)
    )
    x264 = x160 * x263
    x265 = x122 * x252
    x266 = x122 * x257
    x267 = -x227 + x258
    x268 = x113 - x169 - x261 + x262
    x269 = x120 * x266
    x270 = x158 * x257
    x271 = x110 * x123
    x272 = x117 * x271
    x273 = x102 * x123 * x142
    x274 = x118 - x128 - x272 + x273
    x275 = x104 * x271
    x276 = x114 * x241
    x277 = x141 * (
        -x146 * (x147 - x232 - 2 * x272 + 2 * x273)
        - x148 * (x231 - x259)
        - x151 * (x187 + x260 + 2 * x275 - 2 * x276)
    )
    x278 = x160 * x277
    x279 = x122 * x271
    x280 = x228 - x258
    x281 = x169 + x197 + x275 - x276
    x282 = x141 * (-x140 + x153)
    x283 = x160 * x282
    x284 = x122 / x111
    x285 = x109 * x284
    x286 = x110 * x284
    x287 = x139 * x160
    x288 = x105 * x284
    x289 = -x108 * x288
    x290 = x160 * x174
    x291 = -x102 * x288
    x292 = x141 * (-x170 + x171)
    x293 = x160 * x292
    x294 = x106 * x284
    x295 = x160 * x175
    x296 = -x102 * x108 * x284
    x297 = x141 * (-x176 + x177)
    x298 = x160 * x297
    dWdA[0, 0, 0] = (
        x120 * x162
        + x129 * x158
        + x157
        * (
            x116 * x126
            - x119 * x127
            - x128 * x133
            + x137 * x139
            + x140 * x152
            - x152 * x153
        )
        + x159 * x161
        + x164
    )
    dWdA[0, 0, 1] = (
        x108 * x165
        + x116 * x162
        + x137 * x161
        + x157
        * (
            x119 * x168
            - x120 * x126
            + x133 * x169
            - x139 * x159
            + x152 * x170
            - x152 * x171
            + x167
        )
    )
    dWdA[0, 0, 2] = (
        x102 * x165
        + x119 * x162
        + x133 * x160
        + x157
        * (
            -x116 * x168
            + x120 * x127
            - x137 * x175
            + x152 * x176
            - x152 * x177
            + x159 * x174
            + x173
        )
    )
    dWdA[0, 1, 0] = (
        x120 * x191
        + x157
        * (
            x116 * x194
            - x119 * x195
            + x139 * x196
            + x140 * x190
            - x153 * x190
            - x174 * x198
            + x192
        )
        + x158 * x179
        + x161 * x182
    )
    dWdA[0, 1, 1] = (
        x108 * x200
        + x116 * x191
        + x157
        * (
            x119 * x199
            - x120 * x194
            - x139 * x182
            + x170 * x190
            - x171 * x190
            + x175 * x198
        )
        + x161 * x196
        + x164
    )
    dWdA[0, 1, 2] = (
        x102 * x200
        + x119 * x191
        + x157
        * (
            -x116 * x199
            + x120 * x195
            + x174 * x182
            - x175 * x196
            + x176 * x190
            - x177 * x190
            + x201
        )
        + x161 * x198
    )
    dWdA[0, 2, 0] = (
        x120 * x215
        + x157
        * (
            x116 * x217
            - x119 * x218
            + x139 * x220
            + x140 * x214
            - x153 * x214
            + x172
            - x174 * x219
        )
        + x158 * x203
        + x161 * x207
    )
    dWdA[0, 2, 1] = (
        x108 * x221
        + x116 * x215
        + x157
        * (
            x119 * x223
            - x120 * x217
            - x139 * x207
            + x170 * x214
            - x171 * x214
            + x175 * x219
            + x222
        )
        + x161 * x220
    )
    dWdA[0, 2, 2] = (
        x102 * x221
        + x119 * x215
        + x157
        * (
            -x116 * x223
            + x120 * x218
            + x174 * x207
            - x175 * x220
            + x176 * x214
            - x177 * x214
        )
        + x161 * x219
        + x164
    )
    dWdA[1, 0, 0] = (
        x120 * x240
        + x157
        * (x139 * x247 + x140 * x239 - x153 * x239 - x174 * x248 + x243 - x246)
        + x158 * x225
        + x161 * x229
        + x224
    )
    dWdA[1, 0, 1] = (
        x116 * x240
        + x157
        * (
            x119 * x250
            - x120 * x242
            - x139 * x229
            + x170 * x239
            - x171 * x239
            + x175 * x248
            + x192
        )
        + x161 * x247
        + x249
    )
    dWdA[1, 0, 2] = (
        x119 * x240
        + x157
        * (
            -x116 * x250
            + x120 * x245
            + x172
            + x174 * x229
            - x175 * x247
            + x176 * x239
            - x177 * x239
        )
        + x161 * x248
        + x251
    )
    dWdA[1, 1, 0] = (
        x120 * x264
        + x157
        * (
            x116 * x266
            - x119 * x265
            + x139 * x267
            + x140 * x263
            - x153 * x263
            + x167
            - x174 * x268
        )
        + x161 * x256
        + x249
    )
    dWdA[1, 1, 1] = (
        x116 * x264
        + x157
        * (-x139 * x256 + x170 * x263 - x171 * x263 + x175 * x268 + x246 - x269)
        + x158 * x252
        + x161 * x267
        + x224
    )
    dWdA[1, 1, 2] = (
        x119 * x264
        + x157
        * (
            -x116 * x245
            + x120 * x265
            + x174 * x256
            - x175 * x267
            + x176 * x263
            - x177 * x263
            + x222
        )
        + x161 * x268
        + x270
    )
    dWdA[1, 2, 0] = (
        x120 * x278
        + x157
        * (
            x116 * x279
            - x119 * x266
            + x139 * x281
            + x140 * x277
            - x153 * x277
            + x173
            - x174 * x280
        )
        + x161 * x274
        + x251
    )
    dWdA[1, 2, 1] = (
        x116 * x278
        + x157
        * (
            x119 * x242
            - x120 * x279
            - x139 * x274
            + x170 * x277
            - x171 * x277
            + x175 * x280
            + x201
        )
        + x161 * x281
        + x270
    )
    dWdA[1, 2, 2] = (
        x119 * x278
        + x157
        * (x174 * x274 - x175 * x281 + x176 * x277 - x177 * x277 - x243 + x269)
        + x158 * x271
        + x161 * x280
        + x224
    )
    dWdA[2, 0, 0] = x120 * x283 + x157 * (
        x140 * x282 - x153 * x282 + x285 + x286
    )
    dWdA[2, 0, 1] = (
        x116 * x283 + x157 * (x170 * x282 - x171 * x282 + x289) + x287
    )
    dWdA[2, 0, 2] = (
        x119 * x283 + x157 * (x176 * x282 - x177 * x282 + x291) - x290
    )
    dWdA[2, 1, 0] = (
        x120 * x293 + x157 * (x140 * x292 - x153 * x292 + x289) - x287
    )
    dWdA[2, 1, 1] = x116 * x293 + x157 * (
        x170 * x292 - x171 * x292 + x286 + x294
    )
    dWdA[2, 1, 2] = (
        x119 * x293 + x157 * (x176 * x292 - x177 * x292 + x296) + x295
    )
    dWdA[2, 2, 0] = (
        x120 * x298 + x157 * (x140 * x297 - x153 * x297 + x291) + x290
    )
    dWdA[2, 2, 1] = (
        x116 * x298 + x157 * (x170 * x297 - x171 * x297 + x296) - x295
    )
    dWdA[2, 2, 2] = x119 * x298 + x157 * (
        x176 * x297 - x177 * x297 + x285 + x294
    )
    return dWdA

def cross(a, b):
    result = numpy.zeros(3)
    result[0] = a[1] * b[2] - a[2] * b[1]
    result[1] = a[2] * b[0] - a[0] * b[2]
    result[2] = a[0] * b[1] - a[1] * b[0]
    return result

def dWnew(A, B, B0, dist, angle, torsion):
    sin_a = numpy.sin(numpy.pi-angle);
    cos_a = numpy.cos(numpy.pi-angle);
    sin_t = numpy.sin(torsion);
    cos_t = numpy.cos(torsion);
    AB = A-B
    B0B = B0-B
    a_m_b = numpy.sqrt(numpy.dot(AB,AB))
    a_m_b2 = a_m_b*a_m_b
    b0_m_b = numpy.sqrt(numpy.dot(B0B,B0B))
    ABXB0B = cross( AB, B0B );
    abxb0b2 = numpy.dot(ABXB0B,ABXB0B);

    dWdA = numpy.zeros((3, 3, 3))
    dWdA[0, 0, 0] = (
        -ABXB0B[0]*AB[0]*dist*sin_a*sin_t/(a_m_b2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[0]*a_m_b*dist*sin_a*sin_t*(-(-ABXB0B[1]*(2*B0[2] - 2*B[2]) + ABXB0B[2]*(2*B0[1] - 2*B[1]))/(2*a_m_b2) - abxb0b2*(-2*A[0] + 2*B[0])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) - AB[0]*AB[0]*cos_a*dist/(a_m_b2*a_m_b) + 1 - cos_t*dist*sin_a*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])*(-(-ABXB0B[1]*(2*B0[2] - 2*B[2]) + ABXB0B[2]*(2*B0[1] - 2*B[1]))/(2*a_m_b2) - abxb0b2*(-2*A[0] + 2*B[0])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*(AB[1]*B0B[1] + AB[2]*B0B[2])/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*(-2*A[0] + 2*B[0])*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])/(a_m_b2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_a*dist/a_m_b
    )
    dWdA[0, 0, 1] = (
        -ABXB0B[1]*AB[0]*dist*sin_a*sin_t/(a_m_b2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[1]*a_m_b*dist*sin_a*sin_t*(-(-ABXB0B[1]*(2*B0[2] - 2*B[2]) + ABXB0B[2]*(2*B0[1] - 2*B[1]))/(2*a_m_b2) - abxb0b2*(-2*A[0] + 2*B[0])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) - AB[0]*AB[1]*cos_a*dist/(a_m_b2*a_m_b) - B0B[2]*dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])*(-(-ABXB0B[1]*(2*B0[2] - 2*B[2]) + ABXB0B[2]*(2*B0[1] - 2*B[1]))/(2*a_m_b2) - abxb0b2*(-2*A[0] + 2*B[0])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*(2*AB[0]*B0B[1] - AB[1]*B0B[0])/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*(-2*A[0] + 2*B[0])*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])/(a_m_b2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2))
    )
    dWdA[0, 0, 2] = (
        -ABXB0B[2]*AB[0]*dist*sin_a*sin_t/(a_m_b2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[2]*a_m_b*dist*sin_a*sin_t*(-(-ABXB0B[1]*(2*B0[2] - 2*B[2]) + ABXB0B[2]*(2*B0[1] - 2*B[1]))/(2*a_m_b2) - abxb0b2*(-2*A[0] + 2*B[0])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) - AB[0]*AB[2]*cos_a*dist/(a_m_b2*a_m_b) + B0B[1]*dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])*(-(-ABXB0B[1]*(2*B0[2] - 2*B[2]) + ABXB0B[2]*(2*B0[1] - 2*B[1]))/(2*a_m_b2) - abxb0b2*(-2*A[0] + 2*B[0])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*(2*AB[0]*B0B[2] - AB[2]*B0B[0])/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*(-2*A[0] + 2*B[0])*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])/(a_m_b2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2))
    )
    dWdA[0, 1, 0] = (
        -ABXB0B[0]*AB[1]*dist*sin_a*sin_t/(a_m_b2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[0]*a_m_b*dist*sin_a*sin_t*(-(ABXB0B[0]*(2*B0[2] - 2*B[2]) + ABXB0B[2]*(-2*B0[0] + 2*B[0]))/(2*a_m_b2) - abxb0b2*(-2*A[1] + 2*B[1])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) - AB[0]*AB[1]*cos_a*dist/(a_m_b*a_m_b2) + B0B[2]*dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])*(-(ABXB0B[0]*(2*B0[2] - 2*B[2]) + ABXB0B[2]*(-2*B0[0] + 2*B[0]))/(2*a_m_b2) - abxb0b2*(-2*A[1] + 2*B[1])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*(AB[0]*B0B[1] - 2*AB[1]*B0B[0])/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*(-2*A[1] + 2*B[1])*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])/(a_m_b2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2))
    )
    dWdA[0, 1, 1] = (
        -ABXB0B[1]*AB[1]*dist*sin_a*sin_t/(a_m_b2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[1]*a_m_b*dist*sin_a*sin_t*(-(ABXB0B[0]*(2*B0[2] - 2*B[2]) + ABXB0B[2]*(-2*B0[0] + 2*B[0]))/(2*a_m_b2) - abxb0b2*(-2*A[1] + 2*B[1])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) - AB[1]*AB[1]*cos_a*dist/(a_m_b2*a_m_b) + 1 + cos_t*dist*sin_a*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])*(-(ABXB0B[0]*(2*B0[2] - 2*B[2]) + ABXB0B[2]*(-2*B0[0] + 2*B[0]))/(2*a_m_b2) - abxb0b2*(-2*A[1] + 2*B[1])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*(-AB[0]*B0B[0] - AB[2]*B0B[2])/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*(-2*A[1] + 2*B[1])*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])/(a_m_b2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_a*dist/a_m_b
    )
    dWdA[0, 1, 2] = (
        -ABXB0B[2]*AB[1]*dist*sin_a*sin_t/(a_m_b2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[2]*a_m_b*dist*sin_a*sin_t*(-(ABXB0B[0]*(2*B0[2] - 2*B[2]) + ABXB0B[2]*(-2*B0[0] + 2*B[0]))/(2*a_m_b2) - abxb0b2*(-2*A[1] + 2*B[1])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) - AB[1]*AB[2]*cos_a*dist/(a_m_b*a_m_b2) - B0B[0]*dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])*(-(ABXB0B[0]*(2*B0[2] - 2*B[2]) + ABXB0B[2]*(-2*B0[0] + 2*B[0]))/(2*a_m_b2) - abxb0b2*(-2*A[1] + 2*B[1])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*(2*AB[1]*B0B[2] - AB[2]*B0B[1])/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*(-2*A[1] + 2*B[1])*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])/(a_m_b2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2))
    )
    dWdA[0, 2, 0] = (
        -ABXB0B[0]*AB[2]*dist*sin_a*sin_t/(a_m_b2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[0]*a_m_b*dist*sin_a*sin_t*(-(ABXB0B[0]*(-2*B0[1] + 2*B[1]) - ABXB0B[1]*(-2*B0[0] + 2*B[0]))/(2*a_m_b2) - abxb0b2*(-2*A[2] + 2*B[2])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) - AB[0]*AB[2]*cos_a*dist/(a_m_b*a_m_b2) - B0B[1]*dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])*(-(ABXB0B[0]*(-2*B0[1] + 2*B[1]) - ABXB0B[1]*(-2*B0[0] + 2*B[0]))/(2*a_m_b2) - abxb0b2*(-2*A[2] + 2*B[2])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*(AB[0]*B0B[2] - 2*AB[2]*B0B[0])/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*(-2*A[2] + 2*B[2])*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])/(a_m_b2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2))
    )
    dWdA[0, 2, 1] = (
        -ABXB0B[1]*AB[2]*dist*sin_a*sin_t/(a_m_b2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[1]*a_m_b*dist*sin_a*sin_t*(-(ABXB0B[0]*(-2*B0[1] + 2*B[1]) - ABXB0B[1]*(-2*B0[0] + 2*B[0]))/(2*a_m_b2) - abxb0b2*(-2*A[2] + 2*B[2])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) - AB[1]*AB[2]*cos_a*dist/(a_m_b*a_m_b2) + B0B[0]*dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])*(-(ABXB0B[0]*(-2*B0[1] + 2*B[1]) - ABXB0B[1]*(-2*B0[0] + 2*B[0]))/(2*a_m_b2) - abxb0b2*(-2*A[2] + 2*B[2])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*(-AB[1]*B0B[2] + 2*AB[2]*B0B[1])/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*(-2*A[2] + 2*B[2])*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])/(a_m_b2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2))
    )
    dWdA[0, 2, 2] = (
        -ABXB0B[2]*AB[2]*dist*sin_a*sin_t/(a_m_b2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[2]*a_m_b*dist*sin_a*sin_t*(-(ABXB0B[0]*(-2*B0[1] + 2*B[1]) - ABXB0B[1]*(-2*B0[0] + 2*B[0]))/(2*a_m_b2) - abxb0b2*(-2*A[2] + 2*B[2])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) - AB[2]*AB[2]*cos_a*dist/(a_m_b*a_m_b2) + 1 + cos_t*dist*sin_a*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])*(-(ABXB0B[0]*(-2*B0[1] + 2*B[1]) - ABXB0B[1]*(-2*B0[0] + 2*B[0]))/(2*a_m_b2) - abxb0b2*(-2*A[2] + 2*B[2])/(2*a_m_b2*a_m_b2))/(abxb0b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*(-AB[0]*B0B[0] - AB[1]*B0B[1])/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*(-2*A[2] + 2*B[2])*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])/(a_m_b2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_a*dist/a_m_b
    )
    return dWdA

def dWnew2(A, B, B0, dist, angle, torsion):
    #Derivs[s] = Derivs[s].subs(sympy.sqrt((ax-bx)**2+(ay-by)**2+(az-bz)**2), a_m_b)
    #Derivs[s] = Derivs[s].subs(a_m_b**2, a_m_b2)
    #Derivs[s] = Derivs[s].subs(sympy.sqrt((b0x-bx)**2+(b0y-by)**2+(b0z-bz)**2), b0_m_b)
    #Derivs[s] = Derivs[s].subs((ax - bx), ab_x)
    #Derivs[s] = Derivs[s].subs((ay - by), ab_y)
    #Derivs[s] = Derivs[s].subs((az - bz), ab_z)
    #Derivs[s] = Derivs[s].subs((b0x - bx), b0b_x)
    #Derivs[s] = Derivs[s].subs((b0y - by), b0b_y)
    #Derivs[s] = Derivs[s].subs((b0z - bz), b0b_z)
    #Derivs[s] = Derivs[s].subs(ab_y*bb0_z - ab_z*bb0_y, abxb0b_x)
    #Derivs[s] = Derivs[s].subs(ab_z*bb0_x - ab_x*bb0_z, abxb0b_y)
    #Derivs[s] = Derivs[s].subs(ab_x*bb0_y - ab_y*bb0_x, abxb0b_z)
    #Derivs[s] = Derivs[s].subs(abxb0b_x**2 +abxb0b_y**2+abxb0b_z**2, 'abxb0b2')

    sin_a = numpy.sin(numpy.pi-angle);
    cos_a = numpy.cos(numpy.pi-angle);
    sin_t = numpy.sin(torsion);
    cos_t = numpy.cos(torsion);
    AB = A-B
    B0B = B0-B
    a_m_b = numpy.sqrt(numpy.dot(AB,AB))
    a_m_b2 = a_m_b*a_m_b
    b0_m_b = numpy.sqrt(numpy.dot(B0B,B0B))
    ABXB0B = cross( AB, B0B );
    abxb0b2 = numpy.dot(ABXB0B,ABXB0B);

    dwda = numpy.zeros((3, 3, 3))
    x101 = 2*A[0]
    x102 = -x101
    x103 = 2*B[0]
    x104 = x102 + x103
    x105 = 1/abxb0b2
    x106 = 2*B0[1]
    x107 = 2*B[1]
    x108 = -x107
    x109 = 2*B0[2]
    x110 = 2*B[2]
    x111 = -x110
    x112 = x109 + x111
    x113 = -ABXB0B[1]*x112 + ABXB0B[2]*(x106 + x108)
    x114 = 2*A[1]
    x115 = -x114
    x116 = x107 + x115
    x117 = 2*B0[0]
    x118 = x103 - x117
    x119 = ABXB0B[0]*x112 + ABXB0B[2]*x118
    x120 = 2*A[2]
    x121 = x110 - x120
    x122 = -x106
    x123 = ABXB0B[0]*(x107 + x122) - ABXB0B[1]*x118
    x124 = A[1] - B0[1]
    x125 = A[2] - B0[2]
    x126 = x101 - x103
    x127 = -x109 + x120
    x128 = -ABXB0B[1]*x127 + ABXB0B[2]*(x114 + x122)
    x129 = -ABXB0B[2]
    x130 = -A[0] + B0[0]
    x131 = x108 + x114
    x132 = x102 + x117
    x133 = ABXB0B[0]*x127 + ABXB0B[2]*x132
    x134 = -A[1] + B0[1]
    x135 = x111 + x120
    x136 = ABXB0B[0]*(x106 + x115) - ABXB0B[1]*x132
    x137 = -ABXB0B[1]*x121 + ABXB0B[2]*x116
    x138 = ABXB0B[0]*x121 + ABXB0B[2]*x126
    x139 = ABXB0B[0]*x131 - ABXB0B[1]*x126
    dwda[0,0,0] = -ABXB0B[0]*AB[0]*dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[0]*a_m_b*dist*sin_a*sin_t*x105*(-x113/(2*a_m_b2) - abxb0b2*x104/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) - AB[0]**2*cos_a*dist/a_m_b**3 - cos_t*dist*sin_a*x105*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])*(-x113/(2*a_m_b2) - abxb0b2*x104/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + 1 - cos_t*dist*sin_a*(AB[1]*B0B[1] + AB[2]*B0B[2])/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*x104*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2)) + (cos_a*dist)/a_m_b
    dwda[0,0,1] = -ABXB0B[1]*AB[0]*dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[1]*a_m_b*dist*sin_a*sin_t*x105*(-x113/(2*a_m_b2) - abxb0b2*x104/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) - AB[0]*AB[1]*cos_a*dist/a_m_b**3 - B0B[2]*dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*x105*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])*(-x113/(2*a_m_b2) - abxb0b2*x104/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + (cos_t*dist*sin_a/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)))*(2*AB[0]*B0B[1] - AB[1]*B0B[0]) + cos_t*dist*sin_a*x104*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[0,0,2] = -ABXB0B[2]*AB[0]*dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[2]*a_m_b*dist*sin_a*sin_t*x105*(-x113/(2*a_m_b2) - abxb0b2*x104/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) - AB[0]*AB[2]*cos_a*dist/a_m_b**3 + B0B[1]*(dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2))) + cos_t*dist*sin_a*x105*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])*(-x113/(2*a_m_b2) - abxb0b2*x104/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + (cos_t*dist*sin_a/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)))*(2*AB[0]*B0B[2] - AB[2]*B0B[0]) + cos_t*dist*sin_a*x104*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[0,1,0] = -ABXB0B[0]*AB[1]*dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[0]*a_m_b*dist*sin_a*sin_t*x105*(-x119/(2*a_m_b2) - abxb0b2*x116/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) - AB[0]*AB[1]*cos_a*dist/a_m_b**3 + B0B[2]*(dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2))) - cos_t*dist*sin_a*x105*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])*(-x119/(2*a_m_b2) - abxb0b2*x116/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) - cos_t*dist*sin_a*(AB[0]*B0B[1] - 2*AB[1]*B0B[0])/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*x116*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[0,1,1] = -ABXB0B[1]*AB[1]*dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[1]*a_m_b*dist*sin_a*sin_t*x105*(-x119/(2*a_m_b2) - abxb0b2*x116/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) - AB[1]**2*cos_a*dist/a_m_b**3 + cos_t*dist*sin_a*x105*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])*(-x119/(2*a_m_b2) - abxb0b2*x116/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + (cos_t*dist*sin_a/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)))*(-AB[0]*B0B[0] - AB[2]*B0B[2]) + 1 + cos_t*dist*sin_a*x116*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2)) + (cos_a*dist)/a_m_b
    dwda[0,1,2] = -ABXB0B[2]*AB[1]*dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[2]*a_m_b*dist*sin_a*sin_t*x105*(-x119/(2*a_m_b2) - abxb0b2*x116/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) - AB[1]*AB[2]*cos_a*dist/a_m_b**3 - B0B[0]*dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*x105*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])*(-x119/(2*a_m_b2) - abxb0b2*x116/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + (cos_t*dist*sin_a/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)))*(2*AB[1]*B0B[2] - AB[2]*B0B[1]) + cos_t*dist*sin_a*x116*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[0,2,0] = -ABXB0B[0]*AB[2]*dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[0]*a_m_b*dist*sin_a*sin_t*x105*(-x123/(2*a_m_b2) - abxb0b2*x121/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) - AB[0]*AB[2]*cos_a*dist/a_m_b**3 - B0B[1]*dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*x105*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])*(-x123/(2*a_m_b2) - abxb0b2*x121/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) - cos_t*dist*sin_a*(AB[0]*B0B[2] - 2*AB[2]*B0B[0])/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*x121*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[0,2,1] = -ABXB0B[1]*AB[2]*dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[1]*a_m_b*dist*sin_a*sin_t*x105*(-x123/(2*a_m_b2) - abxb0b2*x121/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) - AB[1]*AB[2]*cos_a*dist/a_m_b**3 + B0B[0]*(dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2))) + cos_t*dist*sin_a*x105*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])*(-x123/(2*a_m_b2) - abxb0b2*x121/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + (cos_t*dist*sin_a/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)))*(-AB[1]*B0B[2] + 2*AB[2]*B0B[1]) + cos_t*dist*sin_a*x121*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[0,2,2] = -ABXB0B[2]*AB[2]*dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2)) + ABXB0B[2]*a_m_b*dist*sin_a*sin_t*x105*(-x123/(2*a_m_b2) - abxb0b2*x121/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) - AB[2]**2*cos_a*dist/a_m_b**3 + cos_t*dist*sin_a*x105*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])*(-x123/(2*a_m_b2) - abxb0b2*x121/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + (cos_t*dist*sin_a/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)))*(-AB[0]*B0B[0] - AB[1]*B0B[1]) + 1 + cos_t*dist*sin_a*x121*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2)) + (cos_a*dist)/a_m_b
    dwda[1,0,0] = ABXB0B[0]*a_m_b*dist*sin_a*sin_t*x105*(-x128/(2*a_m_b2) - abxb0b2*x126/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + AB[0]**2*(cos_a*dist/a_m_b**3) - cos_t*dist*sin_a*x105*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])*(-x128/(2*a_m_b2) - abxb0b2*x126/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + (ABXB0B[0]*AB[0])*(dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2))) - cos_t*dist*sin_a*(AB[1]*x124 + AB[2]*x125)/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*x126*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_a*dist/a_m_b
    dwda[1,0,1] = ABXB0B[1]*a_m_b*dist*sin_a*sin_t*x105*(-x128/(2*a_m_b2) - abxb0b2*x126/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + AB[1]*(AB[0]*cos_a*dist/a_m_b**3) + cos_t*dist*sin_a*x105*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])*(-x128/(2*a_m_b2) - abxb0b2*x126/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + (ABXB0B[1]*AB[0])*(dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2))) + (dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2)))*(-A[2] + B0[2]) + cos_t*dist*sin_a*(AB[0]*x124 + x129)/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*x126*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[1,0,2] = ABXB0B[2]*a_m_b*dist*sin_a*sin_t*x105*(-x128/(2*a_m_b2) - abxb0b2*x126/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + AB[2]*(AB[0]*cos_a*dist/a_m_b**3) + cos_t*dist*sin_a*x105*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])*(-x128/(2*a_m_b2) - abxb0b2*x126/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + (ABXB0B[2]*AB[0])*(dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2))) + cos_t*dist*sin_a*(ABXB0B[1] + AB[0]*x125)/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*x126*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2)) + dist*sin_a*sin_t*x124/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[1,1,0] = ABXB0B[0]*a_m_b*dist*sin_a*sin_t*x105*(-x133/(2*a_m_b2) - abxb0b2*x131/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + AB[1]*(AB[0]*cos_a*dist/a_m_b**3) - cos_t*dist*sin_a*x105*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])*(-x133/(2*a_m_b2) - abxb0b2*x131/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + (ABXB0B[0]*AB[1])*(dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2))) - cos_t*dist*sin_a*(AB[1]*x130 + x129)/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*x131*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2)) + dist*sin_a*sin_t*x125/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[1,1,1] = ABXB0B[1]*a_m_b*dist*sin_a*sin_t*x105*(-x133/(2*a_m_b2) - abxb0b2*x131/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + AB[1]**2*(cos_a*dist/a_m_b**3) + cos_t*dist*sin_a*x105*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])*(-x133/(2*a_m_b2) - abxb0b2*x131/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + (ABXB0B[1]*AB[1])*(dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2))) + cos_t*dist*sin_a*(AB[0]*x130 - AB[2]*x125)/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*x131*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_a*dist/a_m_b
    dwda[1,1,2] = ABXB0B[2]*a_m_b*dist*sin_a*sin_t*x105*(-x133/(2*a_m_b2) - abxb0b2*x131/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + cos_t*dist*sin_a*x105*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])*(-x133/(2*a_m_b2) - abxb0b2*x131/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + (ABXB0B[2]*AB[1])*(dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2))) + (AB[1]*AB[2])*(cos_a*dist/a_m_b**3) + cos_t*dist*sin_a*(-ABXB0B[0] + AB[1]*x125)/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*x131*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2)) + dist*sin_a*sin_t*x130/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[1,2,0] = ABXB0B[0]*a_m_b*dist*sin_a*sin_t*x105*(-x136/(2*a_m_b2) - abxb0b2*x135/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + AB[2]*(AB[0]*cos_a*dist/a_m_b**3) - cos_t*dist*sin_a*x105*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])*(-x136/(2*a_m_b2) - abxb0b2*x135/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + (ABXB0B[0]*AB[2])*(dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2))) - cos_t*dist*sin_a*(ABXB0B[1] + AB[2]*x130)/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*x135*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2)) + dist*sin_a*sin_t*x134/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[1,2,1] = ABXB0B[1]*a_m_b*dist*sin_a*sin_t*x105*(-x136/(2*a_m_b2) - abxb0b2*x135/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + cos_t*dist*sin_a*x105*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])*(-x136/(2*a_m_b2) - abxb0b2*x135/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + (ABXB0B[1]*AB[2])*(dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2))) + (AB[1]*AB[2])*(cos_a*dist/a_m_b**3) + (dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2)))*(A[0] - B0[0]) + cos_t*dist*sin_a*(ABXB0B[0] - AB[2]*x134)/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*x135*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[1,2,2] = ABXB0B[2]*a_m_b*dist*sin_a*sin_t*x105*(-x136/(2*a_m_b2) - abxb0b2*x135/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + AB[2]**2*(cos_a*dist/a_m_b**3) + cos_t*dist*sin_a*x105*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])*(-x136/(2*a_m_b2) - abxb0b2*x135/(2*a_m_b2**2))/numpy.sqrt(abxb0b2/a_m_b2) + (ABXB0B[2]*AB[2])*(dist*sin_a*sin_t/(a_m_b**3*numpy.sqrt(abxb0b2/a_m_b2))) + cos_t*dist*sin_a*(AB[0]*x130 + AB[1]*x134)/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*x135*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])/(a_m_b2**2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_a*dist/a_m_b
    dwda[2,0,0] = -ABXB0B[0]*dist*sin_a*sin_t*x105*x137/(2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*x105*x137*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])/(2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*(-AB[1]**2 - AB[2]**2)/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[2,0,1] = -ABXB0B[1]*dist*sin_a*sin_t*x105*x137/(2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) - AB[0]*AB[1]*cos_t*dist*sin_a/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) + AB[2]*(dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2))) - cos_t*dist*sin_a*x105*x137*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])/(2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[2,0,2] = -ABXB0B[2]*dist*sin_a*sin_t*x105*x137/(2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) - AB[0]*AB[2]*cos_t*dist*sin_a/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) - AB[1]*dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*x105*x137*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])/(2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[2,1,0] = -ABXB0B[0]*dist*sin_a*sin_t*x105*x138/(2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) - AB[0]*AB[1]*cos_t*dist*sin_a/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) - AB[2]*dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + cos_t*dist*sin_a*x105*x138*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])/(2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[2,1,1] = -ABXB0B[1]*dist*sin_a*sin_t*x105*x138/(2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + (cos_t*dist*sin_a/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)))*(AB[0]**2 + AB[2]**2) - cos_t*dist*sin_a*x105*x138*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])/(2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[2,1,2] = -ABXB0B[2]*dist*sin_a*sin_t*x105*x138/(2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + AB[0]*(dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2))) - AB[1]*AB[2]*cos_t*dist*sin_a/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*x105*x138*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])/(2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[2,2,0] = -ABXB0B[0]*dist*sin_a*sin_t*x105*x139/(2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) - AB[0]*AB[2]*cos_t*dist*sin_a/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) + AB[1]*(dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2))) + cos_t*dist*sin_a*x105*x139*(-ABXB0B[1]*AB[2] + ABXB0B[2]*AB[1])/(2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[2,2,1] = -ABXB0B[1]*dist*sin_a*sin_t*x105*x139/(2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) - AB[0]*dist*sin_a*sin_t/(a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) - AB[1]*AB[2]*cos_t*dist*sin_a/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)) - cos_t*dist*sin_a*x105*x139*(-ABXB0B[0]*AB[2] + ABXB0B[2]*AB[0])/(2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2))
    dwda[2,2,2] = -ABXB0B[2]*dist*sin_a*sin_t*x105*x139/(2*a_m_b*numpy.sqrt(abxb0b2/a_m_b2)) + (cos_t*dist*sin_a/(a_m_b2*numpy.sqrt(abxb0b2/a_m_b2)))*(AB[0]**2 + AB[1]**2) - cos_t*dist*sin_a*x105*x139*(ABXB0B[0]*AB[1] - ABXB0B[1]*AB[0])/(2*a_m_b2*numpy.sqrt(abxb0b2/a_m_b2))
    return dwda

In [149]:
A=numpy.array([0,0,1])
B=numpy.array([0,0,0])
B0=numpy.array([0,-1,0])
print( dWref(A, B, B0, 1, numpy.pi, 0)[0,:,:])
print( dWnew(A, B, B0, 1, numpy.pi, 0)[0,:,:])
print( dWnew2(A, B, B0, 1, numpy.pi, 0)[0,:,:])


[[2. 0. 0.]
 [0. 2. 0.]
 [0. 0. 1.]]
[[2. 0. 0.]
 [0. 2. 0.]
 [0. 0. 1.]]
[[2. 0. 0.]
 [0. 2. 0.]
 [0. 0. 1.]]
